In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tabulate import tabulate
df=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/최종강남대여소.csv")
df_rental=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020/df_final2020.csv")
df_weather=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020/2020_weather_processed.csv")
df_air=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020/2020_air_processed.csv")
df_people=pd.read_csv(r"/content/drive/MyDrive/Acorn/Project/2020/2020_people_processed.csv")

In [3]:
print('대여소 정보 22 12월 기준')
print(tabulate(df.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print('강남대여소 2020 : {}'.format(len(df_rental['대여 대여소명'].unique())))
print(tabulate(df_rental.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print('강남대여소 공기오염도 2020')
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))
print()
print('유동인구 2020')
print(tabulate(df_people.head(3), headers='keys', tablefmt='psql', showindex=True))
print()

대여소 정보 22 12월 기준
+----+-------------------+-----------------+----------------+
|    |   대여 대여소번호 | 대여 대여소명   | 대여대여소ID   |
|----+-------------------+-----------------+----------------|
|  0 |              2331 | 동영문화센터앞  | ST-814         |
|  1 |              2372 | 대치역 사거리   | ST-1181        |
|  2 |              2413 | 도곡역 1번 출구 | ST-1879        |
+----+-------------------+-----------------+----------------+

강남대여소 2020 : 104
+----+-------------+------------+----------+------------+------------+------------------------+
|    |   기준_날짜 | 대여소ID   |   시간대 |   대여건수 |   반납건수 | 대여 대여소명          |
|----+-------------+------------+----------+------------+------------+------------------------|
|  0 |    20200101 | ST-818     |        0 |          2 |          0 | 3호선 매봉역 3번출구앞 |
|  1 |    20200101 | ST-818     |        1 |          0 |          0 | 3호선 매봉역 3번출구앞 |
|  2 |    20200101 | ST-818     |        2 |          0 |          0 | 3호선 매봉역 3번출구앞 |
+----+-------------+------------+----------


# 대여소+강수량 데이터

In [6]:
# df 데이터프레임의 '대여 대여소명' 열의 앞 뒤 공백 제거
df['대여 대여소명'] = df['대여 대여소명'].str.strip()
df_rental['대여 대여소명'] = df_rental['대여 대여소명'].str.strip()

# df_rental과 df 테이블을 대여 대여소명을 기준으로 조인
df_rental = pd.merge(df_rental, df, on='대여 대여소명', how='left')

print(tabulate(df_rental.head(50), headers='keys', tablefmt='psql', showindex=True))

+----+-------------+------------+----------+------------+------------+------------------------+-------------------+----------------+
|    |   기준_날짜 | 대여소ID   |   시간대 |   대여건수 |   반납건수 | 대여 대여소명          |   대여 대여소번호 | 대여대여소ID   |
|----+-------------+------------+----------+------------+------------+------------------------+-------------------+----------------|
|  0 |    20200101 | ST-818     |        0 |          2 |          0 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |
|  1 |    20200101 | ST-818     |        1 |          0 |          0 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |
|  2 |    20200101 | ST-818     |        2 |          0 |          0 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |
|  3 |    20200101 | ST-818     |        3 |          0 |          2 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |
|  4 |    20200101 | ST-818     |        4 |          0 |          0 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |
|  5 |    2020010

In [8]:
# '일시' 열의 이름을 '날짜'로 변경
df_weather = df_weather.rename(columns={'일시': '날짜'})

# '날짜' 열을 datetime 형식으로 변환
df_weather['날짜'] = pd.to_datetime(df_weather['날짜'])

# 날짜 열을 원하는 형식으로 포맷팅 (년월일)
df_weather['날짜'] = df_weather['날짜'].dt.strftime('%Y%m%d')

# 결과 출력
print(tabulate(df_weather.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['지점', '지점명', '일시', '평균기온(°C)', '일강수량(mm)'], dtype='object')
+----+--------+----------+----------+----------------+----------------+
|    |   지점 | 지점명   |     날짜 |   평균기온(°C) |   일강수량(mm) |
|----+--------+----------+----------+----------------+----------------|
|  0 |    108 | 서울     | 20200101 |           -2.2 |              0 |
|  1 |    108 | 서울     | 20200102 |            1   |              0 |
|  2 |    108 | 서울     | 20200103 |           -0.1 |              0 |
+----+--------+----------+----------+----------------+----------------+


In [9]:
print(df_rental.columns)
df_rental=df_rental.rename(columns={'기준_날짜':'날짜'})
print(df_weather.columns)

Index(['기준_날짜', '대여소ID', '시간대', '대여건수', '반납건수', '대여 대여소명', '대여 대여소번호',
       '대여대여소ID'],
      dtype='object')


In [11]:
print(df_weather.dtypes)
print(df_rental.dtypes)

# '날짜' 열을 int64로 변환
df_weather['날짜'] = df_weather['날짜'].astype('int64')

지점            int64
지점명          object
날짜           object
평균기온(°C)    float64
일강수량(mm)    float64
dtype: object
날짜            int64
대여소ID        object
시간대           int64
대여건수        float64
반납건수        float64
대여 대여소명      object
대여 대여소번호      int64
대여대여소ID      object
dtype: object


In [12]:
# 날짜를 기준으로 두 데이터프레임 병합
merged_df = pd.merge(df_rental, df_weather, on='날짜', how='left')

# 필요한 경우 칼럼명을 조정하여 원하는 형식으로 만들기
merged_df.rename(columns={'일강수량(mm)': '날씨'}, inplace=True)

# 결과 출력
print(merged_df.columns)
print()
print(tabulate(merged_df.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['날짜', '대여소ID', '시간대', '대여건수', '반납건수', '대여 대여소명', '대여 대여소번호', '대여대여소ID',
       '지점', '지점명', '평균기온(°C)', '날씨'],
      dtype='object')

+----+----------+------------+----------+------------+------------+------------------------+-------------------+----------------+--------+----------+----------------+--------+
|    |     날짜 | 대여소ID   |   시간대 |   대여건수 |   반납건수 | 대여 대여소명          |   대여 대여소번호 | 대여대여소ID   |   지점 | 지점명   |   평균기온(°C) |   날씨 |
|----+----------+------------+----------+------------+------------+------------------------+-------------------+----------------+--------+----------+----------------+--------|
|  0 | 20200101 | ST-818     |        0 |          2 |          0 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |    108 | 서울     |           -2.2 |      0 |
|  1 | 20200101 | ST-818     |        1 |          0 |          0 | 3호선 매봉역 3번출구앞 |              2335 | ST-818         |    108 | 서울     |           -2.2 |      0 |
|  2 | 20200101 | ST-818     |        2 |     

In [13]:
# 합친 데이터프레임에서 필요한 칼럼만 가져오기
selected_columns = ['대여소ID','대여 대여소명', '날짜', '시간대', '날씨',  '평균기온(°C)','대여건수', '반납건수','대여 대여소번호']
subset_df = merged_df[selected_columns]
print(tabulate(subset_df.head(3), headers='keys', tablefmt='psql', showindex=True))
print(subset_df.shape)

+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+
|    | 대여소ID   | 대여 대여소명          |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 |
|  2 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        2 |      0 |           -2.2 |          0 |          0 |              2335 |
+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+
(913536, 9)


# 대여소+강수량+(공기오염도 - 평균값을 넣기)

In [14]:
print(df_air.columns)
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['측정일시', '측정소 코드', '측정항목', '평균값', '측정일', '측정시간'], dtype='object')
+----+-------------+---------------+------------+----------+-------------+------------+
|    |    측정일시 |   측정소 코드 |   측정항목 |   평균값 |      측정일 |   측정시간 |
|----+-------------+---------------+------------+----------+-------------+------------|
|  0 | 2.02001e+09 |           123 |          9 |       24 | 2.02001e+07 |        nan |
|  1 | 2.02001e+09 |           123 |          9 |       21 | 2.02001e+07 |          1 |
|  2 | 2.02001e+09 |           123 |          9 |       15 | 2.02001e+07 |          2 |
+----+-------------+---------------+------------+----------+-------------+------------+


In [15]:
# '일시' 열의 이름을 '날짜'로 변경
df_air = df_air.rename(columns={'측정일': '날짜'})

# 결과 출력
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+-------------+---------------+------------+----------+-------------+------------+
|    |    측정일시 |   측정소 코드 |   측정항목 |   평균값 |        날짜 |   측정시간 |
|----+-------------+---------------+------------+----------+-------------+------------|
|  0 | 2.02001e+09 |           123 |          9 |       24 | 2.02001e+07 |        nan |
|  1 | 2.02001e+09 |           123 |          9 |       21 | 2.02001e+07 |          1 |
|  2 | 2.02001e+09 |           123 |          9 |       15 | 2.02001e+07 |          2 |
+----+-------------+---------------+------------+----------+-------------+------------+


In [16]:
# 날짜 형식 변경
df_air['날짜'] = pd.to_datetime(df_air['날짜'], format='%Y%m%d')

# 날짜를 원하는 형식으로 다시 문자열로 변환
df_air['날짜'] = df_air['날짜'].dt.strftime('%Y-%m-%d')

#측정시간 NaN값 0으로
df_air['측정시간'] = df_air['측정시간'].fillna(0)
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+------------+---------------+------------+----------+------------+------------+
|    |   측정일시 |   측정소 코드 |   측정항목 |   평균값 | 날짜       |   측정시간 |
|----+------------+---------------+------------+----------+------------+------------|
|  0 | 2020010100 |           123 |          9 |       24 | 2020-01-01 |          0 |
|  1 | 2020010101 |           123 |          9 |       21 | 2020-01-01 |          1 |
|  2 | 2020010102 |           123 |          9 |       15 | 2020-01-01 |          2 |
+----+------------+---------------+------------+----------+------------+------------+


In [17]:
# '날짜' 열을 datetime 형식으로 변환
df_air['날짜'] = pd.to_datetime(df_air['날짜'])

# 날짜 열을 원하는 형식으로 포맷팅 (년월일)
df_air['날짜'] = df_air['날짜'].dt.strftime('%Y%m%d')

# '날짜' 열을 int64로 변환
df_air['날짜'] = df_air['날짜'].astype('int64')

In [18]:
# 결과 출력
print(tabulate(df_air.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+-------------+---------------+------------+----------+-------------+------------+
|    |    측정일시 |   측정소 코드 |   측정항목 |   평균값 |        날짜 |   측정시간 |
|----+-------------+---------------+------------+----------+-------------+------------|
|  0 | 2.02001e+09 |           123 |          9 |       24 | 2.02001e+07 |          0 |
|  1 | 2.02001e+09 |           123 |          9 |       21 | 2.02001e+07 |          1 |
|  2 | 2.02001e+09 |           123 |          9 |       15 | 2.02001e+07 |          2 |
+----+-------------+---------------+------------+----------+-------------+------------+


In [19]:
# df_air 칼럼의 측정시간 값과 일치하는 것을 기준으로 두 데이터프레임을 날짜를 기준으로 병합
merged_df = pd.merge(subset_df, df_air, left_on=['날짜', '시간대'], right_on=['날짜', '측정시간'], how='left')

# 필요한 경우 칼럼명을 조정하여 원하는 형식으로 만들기
merged_df.rename(columns={'평균값': 'Pm2.5'}, inplace=True)

# 결과 출력
print(tabulate(merged_df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+------------+---------------+------------+---------+------------+
|    | 대여소ID   | 대여 대여소명          |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |   측정일시 |   측정소 코드 |   측정항목 |   Pm2.5 |   측정시간 |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+------------+---------------+------------+---------+------------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 | 2020010100 |           123 |          9 |      24 |          0 |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 | 2020010101 |           123 |          9 |      21 |          1 |
|  2 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |    

In [20]:
# 제외할 열 목록
excluded_columns = ['측정일시', '측정소 코드', '측정항목', '측정시간']

# 제외한 열만 선택하여 새로운 데이터프레임 생성
final_df = merged_df.drop(columns=excluded_columns)
print(tabulate(final_df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+
|    | 대여소ID   | 대여 대여소명          |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |   Pm2.5 |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 |      24 |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 |      21 |
|  2 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        2 |      0 |           -2.2 |          0 |          0 |              2335 |      15 |
+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+


In [21]:
print(tabulate(final_df.head(100), headers='keys', tablefmt='psql', showindex=True))

+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+
|    | 대여소ID   | 대여 대여소명          |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |   Pm2.5 |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 |      24 |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 |      21 |
|  2 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        2 |      0 |           -2.2 |          0 |          0 |              2335 |      15 |
|  3 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        3 |      0 |           -2.2 |          0 |          2 |              2335 |      12 |
|  4 | ST-818     | 3호선 매봉역 

# 유동인구 추가하기

In [22]:
print(df_people.columns)
print(tabulate(df_people.head(3), headers='keys', tablefmt='psql', showindex=True))

Index(['기준일ID', '시간대구분', '총생활인구수', '대여소번호', '보관소(대여소)명'], dtype='object')
+----+------------+--------------+----------------+--------------+-----------------------------------+
|    |   기준일ID |   시간대구분 |   총생활인구수 |   대여소번호 | 보관소(대여소)명                  |
|----+------------+--------------+----------------+--------------+-----------------------------------|
|  0 |   20200101 |            0 |        8313.59 |         2358 | 구룡초사거리 (현대아파트10동 앞 ) |
|  1 |   20200101 |            0 |       25512.9  |         2325 | 대치삼성아파트 앞 교차로          |
|  2 |   20200101 |            0 |       46405.6  |         2343 | 세곡사거리 대왕파출소 앞          |
+----+------------+--------------+----------------+--------------+-----------------------------------+


In [23]:
# '보관소(대여소)명' 열의 이름을 '대여 대여소명'로 변경
df_people = df_people.rename(columns={'보관소(대여소)명': '대여 대여소명','기준일ID':'날짜','시간대구분':'시간대','총생활인구수':'유동인구(명)'})

# '유동인구(명)' 칼럼의 데이터를 int 타입으로 변환
df_people['유동인구(명)'] = df_people['유동인구(명)'].astype(int)

print(tabulate(df_people.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+----------+----------+----------------+--------------+-----------------------------------+
|    |     날짜 |   시간대 |   유동인구(명) |   대여소번호 | 대여 대여소명                     |
|----+----------+----------+----------------+--------------+-----------------------------------|
|  0 | 20200101 |        0 |           8313 |         2358 | 구룡초사거리 (현대아파트10동 앞 ) |
|  1 | 20200101 |        0 |          25512 |         2325 | 대치삼성아파트 앞 교차로          |
|  2 | 20200101 |        0 |          46405 |         2343 | 세곡사거리 대왕파출소 앞          |
+----+----------+----------+----------------+--------------+-----------------------------------+


In [24]:
print(tabulate(final_df.head(3), headers='keys', tablefmt='psql', showindex=True))

+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+
|    | 대여소ID   | 대여 대여소명          |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |   Pm2.5 |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 |      24 |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 |      21 |
|  2 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        2 |      0 |           -2.2 |          0 |          0 |              2335 |      15 |
+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+


In [25]:
# df 데이터프레임의 '대여 대여소명' 열의 앞 뒤 공백 제거
final_df['대여 대여소명'] = final_df['대여 대여소명'].str.strip()
df_people['대여 대여소명'] = df_rental['대여 대여소명'].str.strip()

# df_people 칼럼의 측정시간 값과 일치하는 것을 기준으로 두 데이터프레임을 날짜를 기준으로 병합
final_df = pd.merge(final_df, df_people, left_on=['날짜', '시간대','대여 대여소번호'], right_on=['날짜', '시간대','대여소번호'], how='left')

print(final_df.shape)
print(tabulate(final_df.head(), headers='keys', tablefmt='psql', showindex=True))

(913536, 13)
+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+----------------+--------------+--------------------------------+
|    | 대여소ID   | 대여 대여소명_x        |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |   Pm2.5 |   유동인구(명) |   대여소번호 | 대여 대여소명_y                |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+----------------+--------------+--------------------------------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 |      24 |          27965 |         2335 | K+ 타워 앞                     |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 |      21 |          28337 |         2335 | 강남한양수자인아파트           |

In [26]:
# 제외한 열만 선택하여 새로운 데이터프레임 생성
final_df = final_df.drop(columns='대여 대여소명_y')

# '보관소(대여소)명' 열의 이름을 '대여 대여소명'로 변경
final_df = final_df.rename(columns={'대여 대여소명_x': '대여 대여소명'})

In [27]:
final_df=final_df.drop(columns='대여소번호')

In [28]:
print(tabulate(final_df.head(100), headers='keys', tablefmt='psql', showindex=True))

+----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+----------------+
|    | 대여소ID   | 대여 대여소명          |     날짜 |   시간대 |   날씨 |   평균기온(°C) |   대여건수 |   반납건수 |   대여 대여소번호 |   Pm2.5 |   유동인구(명) |
|----+------------+------------------------+----------+----------+--------+----------------+------------+------------+-------------------+---------+----------------|
|  0 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        0 |      0 |           -2.2 |          2 |          0 |              2335 |      24 |          27965 |
|  1 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        1 |      0 |           -2.2 |          0 |          0 |              2335 |      21 |          28337 |
|  2 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        2 |      0 |           -2.2 |          0 |          0 |              2335 |      15 |          28410 |
|  3 | ST-818     | 3호선 매봉역 3번출구앞 | 20200101 |        3 |      0 |     

In [29]:
final_df.to_csv('/content/drive/MyDrive/Acorn/Project/pleasereallyreally_final2020.csv', index=False)